In [1]:
!pip install nltk
!pip install --user spacy
!pip install pandarallel
!pip install --user https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.0.0/en_core_web_sm-3.0.0.tar.gz

  Using cached pandarallel-1.6.5-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 14.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done


In [2]:
import pandas as pd
import nltk as nltk
import nltk.corpus  
from nltk.text import Text
import matplotlib.pyplot as plt
import seaborn as sns
import re 
import spacy 
from spacy import displacy
import warnings
warnings.filterwarnings("ignore")

#pd.set_option('display.max_rows', 100)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_colwidth', 500)

2023-05-24 23:14:34.805140: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-24 23:14:52.094642: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-05-24 23:14:52.096303: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such 

In [3]:
import multiprocessing
import pandarallel
from pandarallel import pandarallel

In [4]:
# Set up pandarallel
num_processors = multiprocessing.cpu_count()
pandarallel.initialize(nb_workers=num_processors-1, use_memory_fs=False, progress_bar = True)

INFO: Pandarallel will run on 15 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [5]:
nlp = spacy.load("en_core_web_sm")
# Checking active pipeline components
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer']

In [6]:
%%time
df = pd.read_parquet("data_sentiment_topic.parquet")
df.head()

CPU times: user 26.7 s, sys: 5.09 s, total: 31.7 s
Wall time: 23.6 s


,url,date,title,text,main_text,relevant,Tokens,cleaned_tokens,ktrain_topic,vader_score,vader_sentiment,svm_sentiment,vader_score2,textblob_sent,vader_sentiment2
0,http://en.people.cn/n3/2021/0318/c90000-983012...,2021-03-18,Artificial intelligence improves parking effic...,\n\nArtificial intelligence improves parking e...,"Photo taken on July 1, 2019, shows a sign for ...",1,"[Photo, taken, on, July, 1, ,, 2019, ,, shows,...","[Photo, taken, July, show, sign, electronic, t...",5,0.9954,Positive,1,0.9946,Positive,Positive
1,http://newsparliament.com/2020/02/27/children-...,2020-02-27,Children With Autism Saw Their Learning and So...,\nChildren With Autism Saw Their Learning and ...,Children With Autism Saw Their Learning and So...,1,"[Children, With, Autism, Saw, Their, Learning,...","[Children, With, Autism, Saw, Their, Learning,...",5,0.9980,Positive,0,0.9981,Positive,Positive
2,http://www.dataweek.co.za/12835r,2021-03-26,"Forget ML, AI and Industry 4.0 – obsolescence ...","\n\nForget ML, AI and Industry 4.0 – obsolesce...","Forget ML, AI and Industry 4.0 – obsolescence ...",1,"[Forget, ML, ,, AI, and, Industry, 4.0, –, obs...","[Forget, ML, AI, Industry, obsolescence, focus...",5,0.9967,Positive,0,0.9975,Positive,Positive
3,http://www.homeoffice.consumerelectronicsnet.c...,2021-03-10,Strategy Analytics: 71% of Smartphones Sold Gl...,\n\nStrategy Analytics: 71% of Smartphones Sol...,Strategy Analytics: 71% of Smartphones Sold Gl...,1,"[Strategy, Analytics, :, 71, %, of, Smartphone...","[Strategy, Analytics, Smartphones, Sold, Globa...",0,0.9988,Positive,1,0.9988,Positive,Positive
4,http://www.itbusinessnet.com/2020/10/olympus-t...,2020-10-20,Olympus to Support Endoscopic AI Diagnosis Edu...,\n\nOlympus to Support Endoscopic AI Diagnosis...,Olympus to Support Endoscopic AI Diagnosis Edu...,1,"[Olympus, to, Support, Endoscopic, AI, Diagnos...","[Olympus, Support, Endoscopic, AI, Diagnosis, ...",7,0.9923,Positive,0,0.9943,Positive,Positive


## NER using SpaCy without sentence segmentation

In [7]:
# define helper function
def text_ner(text):
    
    entities = []
    labels = []
    
    doc = nlp(text)

    for ent in doc.ents:
        if ent.label_ in ['ORG', 'GPE', 'LOC', 'PERSON']:
            entities.append(ent.text)
            labels.append(ent.label_)
            
    entities_labels_df = pd.DataFrame({"Entities": entities, "Labels": labels})
    
    return entities_labels_df

In [8]:
df['main_text'].values[13]

"Funding boost for artificial intelligence in NHS to speed up diagnosis of deadly diseases — Agenparl UK GOVERNMENT PROVIDES OVER £140 MILLION OF SUPPORT FOR EXPORTS TO GHANAFUNDING BOOST FOR ARTIFICIAL INTELLIGENCE IN NHS TO SPEED UP DIAGNOSIS OF…STRENGTHENING PUBLIC INVESTMENT MANAGEMENT IN THE EASTERN CARIBBEAN CURRENCY UNION: GETTING MORE…TACKLING PRIVATE OVER-INDEBTEDNESS IN ASIA: ECONOMIC AND LEGAL ASPECTSDESTABILIZING STABILITY? EXCHANGE RATE ARRANGEMENTS AND FOREIGN CURRENCY DEBTCOORDINATING REVENUE INCENTIVE POLICIES IN THE CARIBBEANNON-PRIMARY HOME BUYERS, SHADOW BANKING, AND THE US HOUSING MARKETINTELLIGENT EXPORT DIVERSIFICATION: AN EXPORT RECOMMENDATION SYSTEM WITH MACHINE LEARNINGRETHINKING WORK DURING AND AFTER LOCKDOWNTHRIVING IN LATIN AMERICA’S NEXT NORMAL: COMMERCIAL EXCELLENCE IN CPG  Extra £50 million investment in diagnostic centres of excellence which will develop artificial intelligence to diagnose disease Centres will boost diagnostic capabilities, improve outco

In [9]:
%%time
df['NER'] = df['main_text'].parallel_apply(text_ner)

CPU times: user 1min 27s, sys: 11.6 s, total: 1min 39s
Wall time: 26min 16s


In [10]:
df.head()

,url,date,title,text,main_text,relevant,Tokens,cleaned_tokens,ktrain_topic,vader_score,vader_sentiment,svm_sentiment,vader_score2,textblob_sent,vader_sentiment2,NER
0,http://en.people.cn/n3/2021/0318/c90000-983012...,2021-03-18,Artificial intelligence improves parking effic...,\n\nArtificial intelligence improves parking e...,"Photo taken on July 1, 2019, shows a sign for ...",1,"[Photo, taken, on, July, 1, ,, 2019, ,, shows,...","[Photo, taken, July, show, sign, electronic, t...",5,0.9954,Positive,1,0.9946,Positive,Positive,Ent...
1,http://newsparliament.com/2020/02/27/children-...,2020-02-27,Children With Autism Saw Their Learning and So...,\nChildren With Autism Saw Their Learning and ...,Children With Autism Saw Their Learning and So...,1,"[Children, With, Autism, Saw, Their, Learning,...","[Children, With, Autism, Saw, Their, Learning,...",5,0.9980,Positive,0,0.9981,Positive,Positive,E...
2,http://www.dataweek.co.za/12835r,2021-03-26,"Forget ML, AI and Industry 4.0 – obsolescence ...","\n\nForget ML, AI and Industry 4.0 – obsolesce...","Forget ML, AI and Industry 4.0 – obsolescence ...",1,"[Forget, ML, ,, AI, and, Industry, 4.0, –, obs...","[Forget, ML, AI, Industry, obsolescence, focus...",5,0.9967,Positive,0,0.9975,Positive,Positive,Entities Labels 0 Forget...
3,http://www.homeoffice.consumerelectronicsnet.c...,2021-03-10,Strategy Analytics: 71% of Smartphones Sold Gl...,\n\nStrategy Analytics: 71% of Smartphones Sol...,Strategy Analytics: 71% of Smartphones Sold Gl...,1,"[Strategy, Analytics, :, 71, %, of, Smartphone...","[Strategy, Analytics, Smartphones, Sold, Globa...",0,0.9988,Positive,1,0.9988,Positive,Positive,E...
4,http://www.itbusinessnet.com/2020/10/olympus-t...,2020-10-20,Olympus to Support Endoscopic AI Diagnosis Edu...,\n\nOlympus to Support Endoscopic AI Diagnosis...,Olympus to Support Endoscopic AI Diagnosis Edu...,1,"[Olympus, to, Support, Endoscopic, AI, Diagnos...","[Olympus, Support, Endoscopic, AI, Diagnosis, ...",7,0.9923,Positive,0,0.9943,Positive,Positive,E...


In [22]:
import pickle

with open('data_res.pkl', 'wb') as f:
    pickle.dump(df, f)

## Extract NER from each sentence and calculate the sentiment score for these entities
- calculate the sentence sentiment with customized vader

In [6]:
!pip install vaderSentiment

  Using cached vaderSentiment-3.3.2-py2.py3-none-any.whl (125 kB)


In [7]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pickle
import pandas as pd

In [8]:
%%time
with open('data_full.pkl', 'rb') as f:
    df = pickle.load(f)

CPU times: user 1min 27s, sys: 9.7 s, total: 1min 36s
Wall time: 1min 49s


In [9]:
analyzer = SentimentIntensityAnalyzer()
#Update the analyzer with customized word list
pos = ['milestone', 'opportunity', 'growth', 'innovation', 'efficiency', 'productivity', 'competitiveness', 'prospects', 
       'sustainability', 'development', 'progress', 'transformation', 'digitization', 'smart', 'wisdom', 
       'optimization', 'data-driven', 'prediction', 'accuracy', 'high-quality', 'high-efficiency', 'cost-saving', 'safety', 
       'reliability', 'sustainability', 'future-oriented', 'leading', 'leader', 'advantage', 'forward-looking', 
       'exploration', 'creation', 'experimentation', 'discovery', 'breakthrough']

neg = ['unemployment', 'uncertainty', 'contraction', 'reduction', 'downsizing', 'bottleneck', 'challenge', 'obstacle', 'risk', 
       'insecurity', 'instability', 'unreliability', 'opacity', 'restriction', 'limitation', 'regression', 'stagnation', 
       'backwardness', 'failure', 'error', 'defect', 'problem', 'difficulty', 'negative', 'pessimistic', 'worry', 'fear', 
       'inadequacy', 'pressure', 'struggle', 'plight', 'crisis', 'frustration', 'disappointment', 'injury', 'hopelessness', 
       'pessimism', 'downward trend', 'stubbornness', 'conservatism', 'resistance', 'constraint', 'stagnation', 'inflexibility', 
       'outdated', 'slow', 'low-efficiency', 'low-quality', 'high-cost', 'adverse impact', 'loss', 'high risk', 'heavy burden', 
       'drag', 'high challenge', 'detrimental to human beings', 'opposition', 'confrontation']

analyzer.lexicon.update({word: 1.0 for word in pos})
analyzer.lexicon.update({word: -1.0 for word in neg})

In [10]:
analyzer.polarity_scores(df['main_text'].values[0])

{'neg': 0.027, 'neu': 0.858, 'pos': 0.116, 'compound': 0.9946}

In [11]:
def vader_sent(text):
    score = analyzer.polarity_scores(text)
    return score['compound']

In [15]:
# define helper function
def sentence_ner(text):

    entities = []
    labels = []
    score_ls = []
    sentiment = []

    for sent in nltk.sent_tokenize(text):
        doc = nlp(sent)
        for ent in doc.ents:
            if ent.label_ in ['ORG', 'GPE', 'LOC', 'PERSON', 'PRODUCT']:
                entities.append(ent.text)
                labels.append(ent.label_)
                score = vader_sent(sent)
                score_ls.append(score)
                if score > 0: sentiment.append('Positive')
                elif score < 0: sentiment.append('Negative')
                else: sentiment.append('Neutral')
            
    entities_sent_df = pd.DataFrame({"Entities": entities, "Labels": labels, "Score": score_ls, "Sentiment": sentiment})
    
    return entities_sent_df

In [22]:
df.head()

,url,date,title,text,main_text,relevant,Tokens,cleaned_tokens,ktrain_topic,vader_score,vader_sentiment,svm_sentiment,vader_score2,textblob_sent,vader_sentiment2,NER,NER_sent,bert_sentiment
0,http://en.people.cn/n3/2021/0318/c90000-983012...,2021-03-18,Artificial intelligence improves parking effic...,\n\nArtificial intelligence improves parking e...,"Photo taken on July 1, 2019, shows a sign for ...",1,"[Photo, taken, on, July, 1, ,, 2019, ,, shows,...","[Photo, taken, July, show, sign, electronic, t...",5,0.9954,Positive,1,0.9946,Positive,Positive,Ent...,Ent...,positive
1,http://newsparliament.com/2020/02/27/children-...,2020-02-27,Children With Autism Saw Their Learning and So...,\nChildren With Autism Saw Their Learning and ...,Children With Autism Saw Their Learning and So...,1,"[Children, With, Autism, Saw, Their, Learning,...","[Children, With, Autism, Saw, Their, Learning,...",5,0.9980,Positive,0,0.9981,Positive,Positive,E...,E...,positive
2,http://www.dataweek.co.za/12835r,2021-03-26,"Forget ML, AI and Industry 4.0 – obsolescence ...","\n\nForget ML, AI and Industry 4.0 – obsolesce...","Forget ML, AI and Industry 4.0 – obsolescence ...",1,"[Forget, ML, ,, AI, and, Industry, 4.0, –, obs...","[Forget, ML, AI, Industry, obsolescence, focus...",5,0.9967,Positive,0,0.9975,Positive,Positive,Entities Labels 0 Forget...,Entities Labels Sentiment 0 ...,neutral
3,http://www.homeoffice.consumerelectronicsnet.c...,2021-03-10,Strategy Analytics: 71% of Smartphones Sold Gl...,\n\nStrategy Analytics: 71% of Smartphones Sol...,Strategy Analytics: 71% of Smartphones Sold Gl...,1,"[Strategy, Analytics, :, 71, %, of, Smartphone...","[Strategy, Analytics, Smartphones, Sold, Globa...",0,0.9988,Positive,1,0.9988,Positive,Positive,E...,E...,positive
4,http://www.itbusinessnet.com/2020/10/olympus-t...,2020-10-20,Olympus to Support Endoscopic AI Diagnosis Edu...,\n\nOlympus to Support Endoscopic AI Diagnosis...,Olympus to Support Endoscopic AI Diagnosis Edu...,1,"[Olympus, to, Support, Endoscopic, AI, Diagnos...","[Olympus, Support, Endoscopic, AI, Diagnosis, ...",7,0.9923,Positive,0,0.9943,Positive,Positive,E...,E...,neutral


In [23]:
%%time
df['NER_sent'] = df['main_text'].parallel_apply(sentence_ner)

CPU times: user 1min 53s, sys: 21.3 s, total: 2min 14s
Wall time: 1h 44min 11s


In [24]:
#ner = pd.concat(df['NER'].tolist())
sent_ner = pd.concat(df['NER_sent'].tolist())

In [15]:
ner.groupby('Entities').count().sort_values(by = 'Labels', ascending = False).head(10)

,Labels
Entities,
AI,528303
Microsoft,56454
OpenAI,51845
US,40607
India,35519
Artificial Intelligence,35139
China,30424
Google,29150
U.S.,26708


In [16]:
sent_ner.groupby('Entities').count().sort_values(by = 'Labels', ascending = False).head(10)

,Labels,Sentiment
Entities,,
AI,529924,529924
Microsoft,56572,56572
OpenAI,52249,52249
US,40649,40649
India,35667,35667
Artificial Intelligence,35446,35446
China,30455,30455
Google,29226,29226
U.S.,27448,27448


In [17]:
df.head()

,url,date,title,text,main_text,relevant,Tokens,cleaned_tokens,ktrain_topic,vader_score,vader_sentiment,svm_sentiment,vader_score2,textblob_sent,vader_sentiment2,NER,NER_sent
0,http://en.people.cn/n3/2021/0318/c90000-983012...,2021-03-18,Artificial intelligence improves parking effic...,\n\nArtificial intelligence improves parking e...,"Photo taken on July 1, 2019, shows a sign for ...",1,"[Photo, taken, on, July, 1, ,, 2019, ,, shows,...","[Photo, taken, July, show, sign, electronic, t...",5,0.9954,Positive,1,0.9946,Positive,Positive,Ent...,Ent...
1,http://newsparliament.com/2020/02/27/children-...,2020-02-27,Children With Autism Saw Their Learning and So...,\nChildren With Autism Saw Their Learning and ...,Children With Autism Saw Their Learning and So...,1,"[Children, With, Autism, Saw, Their, Learning,...","[Children, With, Autism, Saw, Their, Learning,...",5,0.9980,Positive,0,0.9981,Positive,Positive,E...,E...
2,http://www.dataweek.co.za/12835r,2021-03-26,"Forget ML, AI and Industry 4.0 – obsolescence ...","\n\nForget ML, AI and Industry 4.0 – obsolesce...","Forget ML, AI and Industry 4.0 – obsolescence ...",1,"[Forget, ML, ,, AI, and, Industry, 4.0, –, obs...","[Forget, ML, AI, Industry, obsolescence, focus...",5,0.9967,Positive,0,0.9975,Positive,Positive,Entities Labels 0 Forget...,Entities Labels Sentiment 0 ...
3,http://www.homeoffice.consumerelectronicsnet.c...,2021-03-10,Strategy Analytics: 71% of Smartphones Sold Gl...,\n\nStrategy Analytics: 71% of Smartphones Sol...,Strategy Analytics: 71% of Smartphones Sold Gl...,1,"[Strategy, Analytics, :, 71, %, of, Smartphone...","[Strategy, Analytics, Smartphones, Sold, Globa...",0,0.9988,Positive,1,0.9988,Positive,Positive,E...,E...
4,http://www.itbusinessnet.com/2020/10/olympus-t...,2020-10-20,Olympus to Support Endoscopic AI Diagnosis Edu...,\n\nOlympus to Support Endoscopic AI Diagnosis...,Olympus to Support Endoscopic AI Diagnosis Edu...,1,"[Olympus, to, Support, Endoscopic, AI, Diagnos...","[Olympus, Support, Endoscopic, AI, Diagnosis, ...",7,0.9923,Positive,0,0.9943,Positive,Positive,E...,E...


In [31]:
%%time
import pickle

with open('data_full2.pkl', 'wb') as f:
    pickle.dump(df, f)

CPU times: user 57.4 s, sys: 5.16 s, total: 1min 2s
Wall time: 1min 2s


In [32]:
%%time
with open('data_full2.pkl', 'rb') as f:
    res = pickle.load(f)

CPU times: user 1min 24s, sys: 6 s, total: 1min 30s
Wall time: 1min 30s


In [33]:
res.head()

,url,date,title,text,main_text,relevant,Tokens,cleaned_tokens,ktrain_topic,vader_score,vader_sentiment,svm_sentiment,vader_score2,textblob_sent,vader_sentiment2,NER,NER_sent,bert_sentiment
0,http://en.people.cn/n3/2021/0318/c90000-983012...,2021-03-18,Artificial intelligence improves parking effic...,\n\nArtificial intelligence improves parking e...,"Photo taken on July 1, 2019, shows a sign for ...",1,"[Photo, taken, on, July, 1, ,, 2019, ,, shows,...","[Photo, taken, July, show, sign, electronic, t...",5,0.9954,Positive,1,0.9946,Positive,Positive,Ent...,Ent...,positive
1,http://newsparliament.com/2020/02/27/children-...,2020-02-27,Children With Autism Saw Their Learning and So...,\nChildren With Autism Saw Their Learning and ...,Children With Autism Saw Their Learning and So...,1,"[Children, With, Autism, Saw, Their, Learning,...","[Children, With, Autism, Saw, Their, Learning,...",5,0.9980,Positive,0,0.9981,Positive,Positive,E...,E...,positive
2,http://www.dataweek.co.za/12835r,2021-03-26,"Forget ML, AI and Industry 4.0 – obsolescence ...","\n\nForget ML, AI and Industry 4.0 – obsolesce...","Forget ML, AI and Industry 4.0 – obsolescence ...",1,"[Forget, ML, ,, AI, and, Industry, 4.0, –, obs...","[Forget, ML, AI, Industry, obsolescence, focus...",5,0.9967,Positive,0,0.9975,Positive,Positive,Entities Labels 0 Forget...,Entities Labels Score Sentiment...,neutral
3,http://www.homeoffice.consumerelectronicsnet.c...,2021-03-10,Strategy Analytics: 71% of Smartphones Sold Gl...,\n\nStrategy Analytics: 71% of Smartphones Sol...,Strategy Analytics: 71% of Smartphones Sold Gl...,1,"[Strategy, Analytics, :, 71, %, of, Smartphone...","[Strategy, Analytics, Smartphones, Sold, Globa...",0,0.9988,Positive,1,0.9988,Positive,Positive,E...,E...,positive
4,http://www.itbusinessnet.com/2020/10/olympus-t...,2020-10-20,Olympus to Support Endoscopic AI Diagnosis Edu...,\n\nOlympus to Support Endoscopic AI Diagnosis...,Olympus to Support Endoscopic AI Diagnosis Edu...,1,"[Olympus, to, Support, Endoscopic, AI, Diagnos...","[Olympus, Support, Endoscopic, AI, Diagnosis, ...",7,0.9923,Positive,0,0.9943,Positive,Positive,E...,E...,neutral
